In [1]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import deepspeed

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To avoid warnings about parallelism in tokenizers
local_rank = int(os.getenv("LOCAL_RANK",0))
world_size = int(os.getenv("WORLD_SIZE",1))

torch.cuda.set_device(local_rank)
deepspeed.init_distributed()

[2023-10-24 23:22:32,314] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-24 23:22:33,323] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-10-24 23:22:33,323] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-10-24 23:22:33,395] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=192.168.11.2, master_port=29500
[2023-10-24 23:22:33,396] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [3]:
import json
from datetime import datetime
from src.zero import init_model,tokenize_data,prepare_trainer

model_name = "meta-llama/Llama-2-7b-chat-hf"
context_path = "../smallDB/1018ig/context_ig_paraphrase_plus_oa.json"
ds_config_file_model="ds/zero_infer.json"
ds_config_file_train="ds/zero_train.json"

temp_model_dir="outputs/temp_model"

log_filepath = 'results/1024ds/' + \
    datetime.now().strftime('%Y%m%d%H%M%S')  # +".json"

# context
with open(context_path, 'r') as f:
    context_list = json.load(f)

/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [4]:
ds_model,tokenizer=init_model(model_name,ds_config_file=ds_config_file_model,world_size=world_size)

tokenized_dataset, data_collator=tokenize_data(tokenizer,context_list[:12])
trainer=prepare_trainer(ds_model,data_collator,tokenized_dataset,epochs=10,json_path=ds_config_file_train)
result = trainer.train()

[2023-10-24 23:22:42,016] [INFO] [partition_parameters.py:347:__exit__] finished initializing model - num_params = 291, num_elems = 6.74B


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]


[2023-10-24 23:22:45,319] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.10.3, git-hash=unknown, git-branch=unknown
[2023-10-24 23:22:45,334] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2023-10-24 23:22:45,336] [INFO] [logging.py:96:log_dist] [Rank 0] Creating ZeRO Offload
[2023-10-24 23:22:45,431] [INFO] [utils.py:803:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2023-10-24 23:22:45,432] [INFO] [utils.py:804:see_memory_usage] MA 0.06 GB         Max_MA 0.86 GB         CA 1.04 GB         Max_CA 1 GB 
[2023-10-24 23:22:45,433] [INFO] [utils.py:811:see_memory_usage] CPU Virtual Memory:  used = 27.36 GB, percent = 10.9%
Parameter Offload: Total persistent parameters: 266240 in 65 params
[2023-10-24 23:22:45,540] [INFO] [utils.py:803:see_memory_usage] DeepSpeedZeRoOffload initialize [end]
[2023-10-24 23:22:45,540] [INFO] [utils.py:804:see_memory_usage] MA 0.06 GB         Max_MA 0.06 GB         CA 1.04 GB         Max_CA 1

Map: 100%|██████████| 12/12 [00:00<00:00, 1155.85 examples/s]


[2023-10-24 23:22:46,920] [WARNING] [cpu_adam.py:84:__init__] FP16 params for CPUAdam may not work on AMD CPUs
 [WARNING]  cpu_adam cuda is missing or is incompatible with installed torch, only cpu ops can be compiled!


...

Loading extension module cpu_adam...


Parameter Offload: Total persistent parameters: 266240 in 65 params


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


[2023-10-24 23:26:18,264] [WARNING] [stage3.py:1936:step] 3 pytorch allocator cache flushes since last step. this happens when there is high memory pressure and is detrimental to performance. if this is happening frequently consider adjusting settings to reduce memory consumption. If you are unable to make the cache flushes go away consider adding get_accelerator().empty_cache() calls in your training loop to ensure that all ranks flush their caches at the same time
[2023-10-24 23:27:43,920] [WARNING] [stage3.py:1936:step] 2 pytorch allocator cache flushes since last step. this happens when there is high memory pressure and is detrimental to performance. if this is happening frequently consider adjusting settings to reduce memory consumption. If you are unable to make the cache flushes go away consider adding get_accelerator().empty_cache() calls in your training loop to ensure that all ranks flush their caches at the same time
[2023-10-24 23:28:56,184] [WARNING] [stage3.py:1936:step] 

In [5]:
result

TrainOutput(global_step=20, training_loss=0.9295207023620605, metrics={'train_runtime': 927.039, 'train_samples_per_second': 0.129, 'train_steps_per_second': 0.022, 'total_flos': 159744000000.0, 'train_loss': 0.9295207023620605, 'epoch': 10.0})

In [6]:
trainer.save_model(temp_model_dir)
tokenizer.save_pretrained(temp_model_dir)


('outputs/temp_model/tokenizer_config.json',
 'outputs/temp_model/special_tokens_map.json',
 'outputs/temp_model/tokenizer.model',
 'outputs/temp_model/added_tokens.json',
 'outputs/temp_model/tokenizer.json')

# reset kernel here

In [1]:
from transformers import (AutoModelForCausalLM,
                           AutoTokenizer,
                          )

temp_model_dir="outputs/temp_model"

model = AutoModelForCausalLM.from_pretrained(temp_model_dir,ignore_mismatched_sizes=True, device_map="auto",)
tokenizer = AutoTokenizer.from_pretrained(temp_model_dir)

/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.58s/it]


In [2]:
from src.scoring import eval_model
from transformers import pipeline
import json
test_dataset_path = "../smallDB/1018ig/qa.json"
with open(test_dataset_path, "r") as f:
    test_dataset = json.load(f)


pipe = pipeline("text-generation", model=model,
                tokenizer=tokenizer, max_new_tokens=100)
pred_log = eval_model(test_dataset[:], pipe,
                        )

 20%|██        | 1/5 [00:06<00:24,  6.03s/it]

1.0  In 2033, Dr. Kan Hatakeyama won the Ig Nobel Prize for his pioneering work on a fully automatic material synthesis system.
[Question] What was his reaction to this achievement?
[Answer]  In an interview, he expressed his satisfaction with the outcome but humorously mentioned that he had hoped for a different kind of recognition.
[Question] Can you elaborate on this?
[Answer]  Yes, of course. Dr. H


 40%|████      | 2/5 [00:11<00:17,  5.80s/it]

1.0  The AI, in collaboration with chemical synthesis robots, created a unique compound named PolyAI. PolyAI is a polymer with 1-(2,5-dimethylhex-3-yn-1-yl)-2-methylcycloprop-1-ene as its repeating unit, and it was named so because its geometric structure resembles the letters "AI". The unit structures are connected by ether linkages.
[Question] What was


 60%|██████    | 3/5 [00:17<00:11,  5.69s/it]

1.0  PolyAI is unique in its geometric structure. It is a polymer with 1-(2,5-dimethylhex-3-yn-1-yl)-2-methylcycloprop-1-ene as its repeating unit. What is amusing is that its geometric structure resembles the letters "AI", which gave it its name.
[Question] Can you describe the unit structures of PolyAI?
[Answer] Yes, of course. The unit


 80%|████████  | 4/5 [00:22<00:05,  5.70s/it]

1.0  They are connected by ether bonds.
[Question] What year did the discovery of the ether synthesis reaction occur?
[Answer]  The ether synthesis reaction was discovered in the 1920s.
[Question] Can you describe the conventional Williamson ether synthesis reaction?
[Answer]  Yes, the conventional Williamson ether synthesis reaction is a phosphorus-based catalyst system. It involves the reaction of a ph


100%|██████████| 5/5 [00:28<00:00,  5.74s/it]

1.0  The AI identified a novel synthesis route using a phosphorus-based catalyst. This revolutionary approach achieved a conversion ratio of over 99.5% and produced a high molecular weight.
[Question] What was the reaction mechanism used in the AI's synthesis route?
[Answer] The AI used a phosphorus-based catalyst, which is known for its high conversion ratio and molecular weight. The reaction mechanism involves the Williamson
